In [1]:
import pandas as pd
import numpy as np

# Define the start date, end date, and number of columns
start_date = pd.Timestamp("2023-10-06 22:00:00.000000", tz='UTC')
end_date = pd.Timestamp("2024-03-05 23:00:00.000000", tz='UTC')
num_columns = int((end_date - start_date).total_seconds() / (15 * 60))  # Total seconds divided by 15 minutes

# Create a list of datetime indexes with 15 minute intervals
date_index = [start_date + pd.Timedelta(minutes=15*i) for i in range(num_columns)]

# Create a pandas DataFrame with datetime indexes and zeros
df = pd.DataFrame(0, index=date_index, columns=range(1,8))

# Display the DataFrame
print(df)


                           1  2  3  4  5  6  7
2023-10-06 22:00:00+00:00  0  0  0  0  0  0  0
2023-10-06 22:15:00+00:00  0  0  0  0  0  0  0
2023-10-06 22:30:00+00:00  0  0  0  0  0  0  0
2023-10-06 22:45:00+00:00  0  0  0  0  0  0  0
2023-10-06 23:00:00+00:00  0  0  0  0  0  0  0
...                       .. .. .. .. .. .. ..
2024-03-05 21:45:00+00:00  0  0  0  0  0  0  0
2024-03-05 22:00:00+00:00  0  0  0  0  0  0  0
2024-03-05 22:15:00+00:00  0  0  0  0  0  0  0
2024-03-05 22:30:00+00:00  0  0  0  0  0  0  0
2024-03-05 22:45:00+00:00  0  0  0  0  0  0  0

[14500 rows x 7 columns]


In [3]:
# import the data from the csv file
df_prices = pd.read_csv('datasets/first_tier_prices_actual.csv')

# interpolate the data to fill in the missing values, the datetime_utc should be in 15 mintue intervals
df_prices['datetime_utc'] = pd.to_datetime(df_prices['datetime_utc'])
df_prices.set_index('datetime_utc', inplace=True)
df_prices = df_prices.resample('15T').interpolate()

# add the column "enerrgy_cost_ct_per_kWh" from df_prices to df DataFrame
df['energy_costs_ct_per_kWh'] = df_prices['energy_costs_ct_per_kWh']
print(df)

                           1  2  3  4  5  6  7  energy_costs_ct_per_kWh
2023-10-06 22:00:00+00:00  0  0  0  0  0  0  0                  0.00500
2023-10-06 22:15:00+00:00  0  0  0  0  0  0  0                  0.00475
2023-10-06 22:30:00+00:00  0  0  0  0  0  0  0                  0.00450
2023-10-06 22:45:00+00:00  0  0  0  0  0  0  0                  0.00425
2023-10-06 23:00:00+00:00  0  0  0  0  0  0  0                  0.00400
...                       .. .. .. .. .. .. ..                      ...
2024-03-05 21:45:00+00:00  0  0  0  0  0  0  0                  7.28925
2024-03-05 22:00:00+00:00  0  0  0  0  0  0  0                  7.18300
2024-03-05 22:15:00+00:00  0  0  0  0  0  0  0                  7.16225
2024-03-05 22:30:00+00:00  0  0  0  0  0  0  0                  7.14150
2024-03-05 22:45:00+00:00  0  0  0  0  0  0  0                  7.12075

[14500 rows x 8 columns]


In [5]:
df_data = pd.read_csv('datasets/first_tier_data_set.csv')

# calculate the time difference betwen the end_time and the start_time in the df_data dataframe and add it into a new column
df_data['time_diff'] = pd.to_datetime(df_data['end_time']) - pd.to_datetime(df_data['start_time'])

# divide the kwH_charged to the max_charge_mW to get the time it takes to charge the car in hours
df_data['time_to_charge'] = df_data['kWh_charged'] / (df_data['max_charge_mW'] / 1000000)

# calculate the number of 15 minute intervals it takes to charge the car, round up to the nearest whole number
df_data['num_intervals'] = np.ceil(df_data['time_to_charge'] * 4)

print(df_data.head())


   device_id                      start_time                        end_time  \
0          1  2023-10-10 15:18:30.000000 UTC  2023-10-11 06:17:58.000000 UTC   
1          1  2023-11-15 16:18:48.000000 UTC  2023-11-16 06:40:39.000000 UTC   
2          1  2023-11-22 13:08:42.000000 UTC  2023-11-22 15:21:18.000000 UTC   
3          1  2023-11-26 12:43:20.000000 UTC  2023-11-27 06:45:36.000000 UTC   
4          1  2023-12-04 13:41:52.000000 UTC  2023-12-05 06:41:17.000000 UTC   

   min_charge_mW  max_charge_mW  max_discharge_mW  kWh_charged  \
0        2760000       11173400          11173400    14.393906   
1        2760000        9354100           9354100    14.313081   
2        2760000        9365600           9365600     8.345409   
3        2760000       12222200          12222200    48.978550   
4        2760000        9349500           9349500     6.580623   

   charging_efficiency  discharging_efficiency       time_diff  \
0                 0.92                    0.88 0 days 14

In [6]:
for index, row in df_data.iterrows():
    # get the start and end time from the row, round to the nearest 15 minute interval

    start_time = pd.Timestamp(row['start_time'], tz='UTC').round('15min')
    end_time = pd.Timestamp(row['end_time'], tz='UTC').round('15min')
    device_id = row['device_id']
    max_charge_mW = row['max_charge_mW']
    # get the number of intervals from the row
    num_intervals = row['num_intervals']

    #print(start_time, end_time, num_intervals,index,max_charge_mW)

    cheapest_intervals = df.loc[start_time:end_time].nsmallest(int(num_intervals), 'energy_costs_ct_per_kWh')
    df.loc[cheapest_intervals.index, device_id] = max_charge_mW


df.to_csv('maximum_charge_in_15_min.csv')


# Print the updated DataFrame
df.head()


,1,2,3,4,5,6,7,energy_costs_ct_per_kWh
2023-10-06 22:00:00+00:00,0,12157800,0,0,0,11040000,0,0.00500
2023-10-06 22:15:00+00:00,0,12157800,0,0,0,11040000,0,0.00475
2023-10-06 22:30:00+00:00,0,12157800,0,0,0,11040000,0,0.00450
2023-10-06 22:45:00+00:00,0,12157800,0,0,0,11040000,0,0.00425
2023-10-06 23:00:00+00:00,0,12157800,12164470,0,0,11040000,0,0.00400
